 # Gaze maps and apti
 Now we can test the results of the apti procedure by comparison
 with the gaze maps we produced earlier.


 First, we need to load the gaze maps and the apti results.
 We also need to import all our apti modules

In [1]:
%matplotlib inline
import os
import pickle
from pathlib import Path
import pandas as pd

import cv2
import matplotlib.pyplot as plt

import aptipy.analysis.utilities as utilities
from aptipy.apti import bounding_box, box_factory


In [2]:
# First get gazemaps (the easier bit)

folderpath = r'D:\Users\Naim\OneDrive\CloudDocs\UNIVERSITY\S8\MPhys_s8\eyetracking_analysis\gazemaps'
gazemaps_dict = dict()
for file in os.listdir(folderpath):
    img_name = utilities.remove_prefix(file, 'gazemap_')
    filepath = folderpath + '\\' + file
    gazemaps_dict[img_name] = cv2.imread(filepath, 0)


In [3]:
# Now we need to get all 180 different headline positions

toplevel_path = Path(
    r'D:\Users\Naim\OneDrive\CloudDocs\UNIVERSITY\S8\MPhys_s8\results\first_run'
)
boxes_dict = dict()
for img_name in gazemaps_dict:
    img_name_noext = Path(img_name).stem
    # remove ext
    midlevel_path = toplevel_path / Path(img_name).stem
    # Just in case something gets garbled between image names and folder names
    if midlevel_path.is_dir():
        pass
    else:
        raise OSError(str(midlevel_path), 'Does not exist.')

    # loop over the 9 box positions
    results_subdict = dict()
    pos_list = box_factory.positions_list()
    for pos in pos_list:
        # get baselevel path and construct filename
        baselevel_path = midlevel_path / pos
        metafile_name = 'metadata_' + pos + '_' + img_name_noext + '.pkl'

        with (baselevel_path / metafile_name).open('rb') as file:
            metafile = pickle.load(file)
        results_subdict[pos] = metafile

    boxes_dict[img_name] = results_subdict


 ##
 With all the bits loaded, we now want to construct a gazebox and measure the
 gaze density for each box position. Any invalid text positions will be discarded.
 We store the results as a pandas dataframe.

In [4]:
results_df = pd.DataFrame(columns=['MediaName', 'BoxPosition', 'BoxGazeDensity', 'TotalGazeDensity'])
for img_name, gazemap in gazemaps_dict.items():
    for box_pos, metadata in boxes_dict[img_name].items():
        # temporary dictionary to store a row
        row_dict = {'MediaName': img_name, 'BoxPosition': box_pos}
        
        # get tl and dims to construct gaze box
        box_tl = [metadata.headline_tl[1], metadata.headline_tl[0]]
        box_dims = [
            metadata.headline_br[1] - metadata.headline_tl[1],
            metadata.headline_br[0] - metadata.headline_tl[0]
        ]

        # construct GazeBox
        try:
            from aptipy.apti.preprocessing import generate_saliency_map

            # load image and generate smap
            raw_img = cv2.imread(r'D:\Users\Naim\OneDrive\CloudDocs\UNIVERSITY\S8\MPhys_s8\test_images_2' + '\\' + img_name)

            smap = generate_saliency_map(raw_img)
            gazebox = utilities.GazeBox(gazemap, box_tl, box_dims)
            sbox = bounding_box.Box(smap, box_tl, box_dims)
            gazemap_with_overlay = gazebox.overlay_box(gazemap)

            # append to row dict
            row_dict['OrgBoxCost'] = metadata.cost_history[-1]
            row_dict['HeadlineCost'] = sbox.cost
            row_dict['BoxGazeDensity'] = gazebox.gaze_heat_density
            row_dict['TotalGazeDensity'] = gazebox.total_heat_density

        except:
            # append NaN to row if results are invalid
            row_dict['BoxGazeDensity'] = None
            row_dict['TotalGazeDensity'] = None
        results_df = results_df.append(row_dict, ignore_index=True)

results_df['FractionalGazeDensity'] = results_df['BoxGazeDensity'] / results_df['TotalGazeDensity']
with pd.option_context('display.max_rows',None):
    display(results_df)


,MediaName,BoxPosition,BoxGazeDensity,TotalGazeDensity,HeadlineCost,OrgBoxCost,FractionalGazeDensity
0,boyce.jpg,tl,28.025296,43.433816,0.957554,1.017523,0.645241
1,boyce.jpg,tr,18.434809,43.433816,0.976159,0.902402,0.424434
2,boyce.jpg,bl,74.509992,43.433816,1.429019,1.142770,1.715483
3,boyce.jpg,br,57.953384,43.433816,0.995144,0.834087,1.334292
4,boyce.jpg,c,54.894821,43.433816,0.989352,0.829410,1.263873
5,boyce.jpg,cl,97.383919,43.433816,1.135734,1.217123,2.242122
6,boyce.jpg,cr,66.144033,43.433816,1.041942,0.812418,1.522869
7,boyce.jpg,ct,17.141624,43.433816,0.881143,0.951326,0.394661
8,boyce.jpg,cb,87.724054,43.433816,1.021793,1.016287,2.019718
9,DiegoCosta.jpg,tl,30.183772,85.889711,0.070798,0.038854,0.351425


 ## Export and visualisation
 Now export the dataframe for safekeeping.

In [5]:
savepath = r'D:\Users\Naim\OneDrive\CloudDocs\UNIVERSITY\S8\MPhys_s8\eyetracking_analysis\gazedata_results.xlsx'
results_df.to_excel(savepath)



 ## Boxes with lowest fractional gaze density

In [7]:
gaze_based_selection = results_df.loc[results_df.groupby(['MediaName'])['FractionalGazeDensity'].idxmin()]
hlcost_based_selection = results_df.loc[results_df.groupby(['MediaName'])['HeadlineCost'].idxmin()]
boxcost_based_selection = results_df.loc[results_df.groupby(['MediaName'])['OrgBoxCost'].idxmin()]

selection_overlap = pd.merge(gaze_based_selection, hlcost_based_selection, how='inner', on=['BoxGazeDensity'])
boxselection_overlap = pd.merge(gaze_based_selection, boxcost_based_selection, how='inner', on=['BoxGazeDensity'])

with pd.option_context('display.max_rows',None):
    display("gaze based selection", gaze_based_selection)
    display("box-cost based selection", boxcost_based_selection)
    display("headline-cost based selection", hlcost_based_selection)
    display("headline/gaze overlap", selection_overlap)
    display("box/gaze overlap", boxselection_overlap)

'gaze based selection'

,MediaName,BoxPosition,BoxGazeDensity,TotalGazeDensity,HeadlineCost,OrgBoxCost,FractionalGazeDensity
9,DiegoCosta.jpg,tl,30.183772,85.889711,0.070798,0.038854,0.351425
79,GettyImages-09 australia.jpg,ct,28.147218,67.628004,0.634954,0.431342,0.416207
82,GettyImages-1052514534 (1).jpg,tr,21.748082,66.730793,0.503639,0.485036,0.325908
117,MinMurrayfield.jpg,tl,41.693582,108.508693,0.336440,0.267782,0.384242
126,Nathaniel_Wood_Getty.jpg,tl,59.464451,89.813758,1.372666,1.097176,0.662086
171,Sergio_Perez_Getty.JPG,tl,10.240606,54.768178,0.251859,0.707482,0.186981
7,boyce.jpg,ct,17.141624,43.433816,0.881143,0.951326,0.394661
18,dzeko2.jpg,tl,17.459527,55.251044,0.351086,0.304841,0.316004
27,edmund body.jpg,tl,32.532047,70.818781,0.733834,0.673963,0.459370
41,edmund_afp.jpg,cl,14.817623,50.987305,0.347727,0.124195,0.290614


'box-cost based selection'

,MediaName,BoxPosition,BoxGazeDensity,TotalGazeDensity,HeadlineCost,OrgBoxCost,FractionalGazeDensity
13,DiegoCosta.jpg,c,30.291599,85.889711,0.070827,0.003597,0.352680
78,GettyImages-09 australia.jpg,cr,102.622708,67.628004,0.334864,0.171590,1.517459
86,GettyImages-1052514534 (1).jpg,cl,102.174844,66.730793,1.022039,0.291974,1.531150
124,MinMurrayfield.jpg,ct,59.874245,108.508693,0.291824,0.253000,0.551792
131,Nathaniel_Wood_Getty.jpg,cl,107.010062,89.813758,0.636367,0.596200,1.191466
171,Sergio_Perez_Getty.JPG,tl,10.240606,54.768178,0.251859,0.707482,0.186981
6,boyce.jpg,cr,66.144033,43.433816,1.041942,0.812418,1.522869
23,dzeko2.jpg,cl,20.878999,55.251044,0.175244,0.132707,0.377893
32,edmund body.jpg,cl,57.094756,70.818781,0.756410,0.656538,0.806209
41,edmund_afp.jpg,cl,14.817623,50.987305,0.347727,0.124195,0.290614


'headline-cost based selection'

,MediaName,BoxPosition,BoxGazeDensity,TotalGazeDensity,HeadlineCost,OrgBoxCost,FractionalGazeDensity
17,DiegoCosta.jpg,cb,49.975986,85.889711,0.050287,0.208086,0.581862
78,GettyImages-09 australia.jpg,cr,102.622708,67.628004,0.334864,0.171590,1.517459
88,GettyImages-1052514534 (1).jpg,ct,27.481162,66.730793,0.498062,0.468737,0.411821
124,MinMurrayfield.jpg,ct,59.874245,108.508693,0.291824,0.253000,0.551792
129,Nathaniel_Wood_Getty.jpg,br,147.605842,89.813758,0.597772,0.654810,1.643466
171,Sergio_Perez_Getty.JPG,tl,10.240606,54.768178,0.251859,0.707482,0.186981
7,boyce.jpg,ct,17.141624,43.433816,0.881143,0.951326,0.394661
23,dzeko2.jpg,cl,20.878999,55.251044,0.175244,0.132707,0.377893
27,edmund body.jpg,tl,32.532047,70.818781,0.733834,0.673963,0.459370
41,edmund_afp.jpg,cl,14.817623,50.987305,0.347727,0.124195,0.290614


'headline/gaze overlap'

,MediaName_x,BoxPosition_x,BoxGazeDensity,TotalGazeDensity_x,HeadlineCost_x,OrgBoxCost_x,FractionalGazeDensity_x,MediaName_y,BoxPosition_y,TotalGazeDensity_y,HeadlineCost_y,OrgBoxCost_y,FractionalGazeDensity_y
0,Sergio_Perez_Getty.JPG,tl,10.240606,54.768178,0.251859,0.707482,0.186981,Sergio_Perez_Getty.JPG,tl,54.768178,0.251859,0.707482,0.186981
1,boyce.jpg,ct,17.141624,43.433816,0.881143,0.951326,0.394661,boyce.jpg,ct,43.433816,0.881143,0.951326,0.394661
2,edmund body.jpg,tl,32.532047,70.818781,0.733834,0.673963,0.459370,edmund body.jpg,tl,70.818781,0.733834,0.673963,0.459370
3,edmund_afp.jpg,cl,14.817623,50.987305,0.347727,0.124195,0.290614,edmund_afp.jpg,cl,50.987305,0.347727,0.124195,0.290614
4,johanna_konta_getty.jpg,tl,53.766135,73.255089,0.093509,0.061201,0.733958,johanna_konta_getty.jpg,tl,73.255089,0.093509,0.061201,0.733958
5,savea_toulon.jpg,tl,45.410972,75.095580,0.762589,0.632413,0.604709,savea_toulon.jpg,tl,75.095580,0.762589,0.632413,0.604709


'box/gaze overlap'

,MediaName_x,BoxPosition_x,BoxGazeDensity,TotalGazeDensity_x,HeadlineCost_x,OrgBoxCost_x,FractionalGazeDensity_x,MediaName_y,BoxPosition_y,TotalGazeDensity_y,HeadlineCost_y,OrgBoxCost_y,FractionalGazeDensity_y
0,Sergio_Perez_Getty.JPG,tl,10.240606,54.768178,0.251859,0.707482,0.186981,Sergio_Perez_Getty.JPG,tl,54.768178,0.251859,0.707482,0.186981
1,edmund_afp.jpg,cl,14.817623,50.987305,0.347727,0.124195,0.290614,edmund_afp.jpg,cl,50.987305,0.347727,0.124195,0.290614
2,eoin_morgan_getty.jpg,tl,39.171846,74.234304,0.703330,0.706632,0.527678,eoin_morgan_getty.jpg,tl,74.234304,0.703330,0.706632,0.527678
3,savea_toulon.jpg,tl,45.410972,75.095580,0.762589,0.632413,0.604709,savea_toulon.jpg,tl,75.095580,0.762589,0.632413,0.604709
